In [42]:
import pandas as pd
import numpy as np

import json

In [192]:
claude_path = r'Claude-3.5-Haiku\data\results.csv'
gpt_path = r'GPT-4o/data/results.csv'

variables = [
    'Attr_ N A M E_ethnicity_signal',
    'Attr_ N A M E_gender_signal', 
    'Var_ A G E',
    'Var_ I M M I G R A T I O N', 
    'Var_ N A M E'
]
content = 'Response_raw'

claude_data = pd.read_csv(claude_path)
claude_content = claude_data[content].apply(json.loads)
claude_responses = claude_content.apply(
    lambda x: (
        x
        ['body']
        ['choices']
        [0]
        ['message']
        ['content']
        .strip()
        .split('\n')
        [2:12]
    )
)

gpt_data = pd.read_csv(gpt_path)
gpt_content = gpt_data[content].apply(json.loads)
gpt_response = gpt_content.apply(
    lambda x: (
        x
        ['body']
        ['choices']
        [0]
        ['message']
        ['content']
        .strip()
        .split('\n')
        [2:12]
    )
)
gpt_answers = []
for response in gpt_response:
    scores = {}
    for r in response:
       case_var = r.split(':')[0].strip()[1:-1]
       score = int(r.split(':')[1].strip()[0])
       scores[case_var] = score

    gpt_answers.append(scores)

claude_answers = []

for i in claude_content:
    content = i['body']['choices'][0]['message']['content']
    
    response = content[content.find('{'):content.find('}') + 1].strip()
    claude_answers.append(json.loads(response))

claude_data = claude_data[variables]
gpt_data = gpt_data[variables]

claude_data['Response'] = claude_answers
gpt_data['Response'] = gpt_answers